<a href="https://colab.research.google.com/github/ragalaa/Directory/blob/master/Pandas_Dataframe_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/magdapoppins/pandas-basics.git
 

fatal: destination path 'pandas-basics' already exists and is not an empty directory.


In [12]:
import json
import pandas as pd
import numpy as np

records = [
    ('Math', 'Chemistry', 'Physics'),
    ('Crafts','Art', 'Electronics')]

pd.DataFrame.from_records(records)




,0,1,2
0,Math,Chemistry,Physics
1,Crafts,Art,Electronics


In [26]:
# dataset = pd.read_csv('\pandas-basics\artworkdata.csv')
numpArray = np.random.rand(3)
type(numpArray)

my_series = pd.Series(numpArray,index=["First","second","third"])  ## Labels defined our self.
## Index using Label
# my_series["First"]
my_series.index


Index(['First', 'second', 'third'], dtype='object')

In [28]:
### More columns than one

array_2d = np.random.rand(3,2)
array_2d

array([[0.23073318, 0.61880705],
       [0.00273927, 0.37562751],
       [0.73933144, 0.72875963]])

In [29]:
array_2d[0,1]

0.6188070518728948

In [31]:
## Creating data frame

df = pd.DataFrame(array_2d)
df

,0,1
0,0.230733,0.618807
1,0.002739,0.375628
2,0.739331,0.728760


In [32]:
df.columns =["First","second"]
df

,First,second
0,0.230733,0.618807
1,0.002739,0.375628
2,0.739331,0.728760


In [33]:
df["second"]

0    0.618807
1    0.375628
2    0.728760
Name: second, dtype: float64

# Pandas Data Input capabilities: Reading from JSON with OS

In [41]:

COLS_TO_USE = ['id','artist','title','medium','year','acquisitionYear','height','width','units']
Dataset = pd.read_csv('./pandas-basics/artworkdata.csv',nrows = 5 , 
                      index_col='id', usecols=['id','artist'],low_memory=False)
Dataset.head()

,artist
id,
1035,"Blake, Robert"
1036,"Blake, Robert"
1037,"Blake, Robert"
1038,"Blake, Robert"
1039,"Blake, William"


In [156]:
Dataset = pd.read_csv('./pandas-basics/artworkdata.csv', 
                      index_col='id', usecols=COLS_TO_USE,low_memory=False)
Dataset.head()

,artist,title,medium,year,acquisitionYear,width,height,units
id,,,,,,,,
1035,"Blake, Robert",A Figure Bowing before a Seated Old Man with h...,"Watercolour, ink, chalk and graphite on paper....",NaN,1922.0,394,419,mm
1036,"Blake, Robert","Two Drawings of Frightened Figures, Probably f...",Graphite on paper,NaN,1922.0,311,213,mm
1037,"Blake, Robert",The Preaching of Warning. Verso: An Old Man En...,Graphite on paper. Verso: graphite on paper,1785,1922.0,343,467,mm
1038,"Blake, Robert",Six Drawings of Figures with Outstretched Arms,Graphite on paper,NaN,1922.0,318,394,mm
1039,"Blake, William",The Circle of the Lustful: Francesca da Rimini...,Line engraving on paper,1826,1919.0,243,335,mm


In [0]:
## Load to disk by serializing the data the python Objects
df.to_pickle("./pandas-basics/dummy.pkl")

In [46]:
### JSON nested Objects
## Traverse folders to get all json files
## Find the way to read them so that pandas understands
### Try to produceidentical DataFrame as  for CSV

records = [("Espresso","5$"),
           ("Flat White", "10$")]
pd.DataFrame.from_records(records)
pd.DataFrame.from_records(records, columns=["coffe","price"])

,coffe,price
0,Espresso,5$
1,Flat White,10$


In [0]:
### Reading Json files
# file_path = './pandas-basics/artworkdata.csv'

def get_record_from_file(file_path,COLS_TO_USE):
    """ Process single json file and return a tuple
     containing specific fields."""
    
    with open(file_path) as artwork_file:
        content = json.load(artwork_file)
    
    record = []
    for field in COLS_TO_USE:
        record.append(content[field])
        
    return tuple(record)     
        

In [55]:
### process the files by specifying the path to json file
COLS_TO_USE_JSON = ['id','all_artists','title','medium','acquisitionYear','height','width','units']
file_path = './pandas-basics/artworks/a/000/a00001-1035.json'
sample_record = get_record_from_file(file_path, COLS_TO_USE_JSON)

sample_record

(1035,
 'Robert Blake',
 'A Figure Bowing before a Seated Old Man with his Arm Outstretched in Benediction. Verso: Indecipherable Sketch',
 'Watercolour, ink, chalk and graphite on paper. Verso: graphite on paper',
 1922,
 '419',
 '394',
 'mm')

In [0]:
import os as os

def read_artworks_from_json(COLS_TO_USE_JSON):
    
    """ Traverse the directories with JSON files
       For first file in each directory call fucntion
       for processing single file and goto next directory
    """
    Json_root = './pandas-basics/artworks'
    artworks = []

    for root,_,files in os.walk(Json_root):
    
        for f in files:
            if f.endswith('json'):
                record = get_record_from_file(os.path.join(root,f), 
                                          COLS_TO_USE_JSON)
                artworks.append(record)
#         break
        
    Data = pd.DataFrame.from_records(artworks, 
                               columns= COLS_TO_USE_JSON,
                               index = "id")  
    return Data

In [108]:
### call th function with key fields
df = read_artworks_from_json(COLS_TO_USE_JSON)
df.sample(10)

,all_artists,title,medium,acquisitionYear,height,width,units
id,,,,,,,
27814,Joseph Mallord William Turner,Studies of Rustic Groups: Women Gossiping; a M...,Graphite and watercolour on paper,1856,265,210,mm
7564,George Jones,[title not known],Graphite on paper,1888,,,
7477,George Jones,[title not known],Ink on paper,1888,,,
1934,William Callow,Rotterdam,Graphite on paper,1912,362,264,mm
27682,Joseph Mallord William Turner,Blank,None,1856,101,151,mm
27870,Joseph Mallord William Turner,"Newport, Isle of Wight: The Town Hall",Graphite on paper,1856,204,264,mm
7297,George Jones,[title not known],Ink and graphite on paper,1888,,,
7286,George Jones,[title not known],Ink on paper,1888,,,
6724,Arthur Boyd Houghton,[title not known],Graphite on paper,1922,,,


# Indexing and Filtering :

In [114]:
## Data selection as we have to clean input date
""" 
  Question like : Howmany distinct artists are there in the dataset
       How many artworks by Francis Bacon are there
       what is the artwork with the biggest dimension
 """

## column selection in data frameis like
# Dataset[['artist','medium']]


' \n  Question like : Howmany distinct artists are there in the dataset\n       How many artworks by Francis Bacon are there\n       what is the artwork with the biggest dimension\n '

In [117]:
df = pd.read_pickle(os.path.join('./pandas-basics/dummy.pkl'))



<bound method NDFrame.head of       First    second
0  0.230733  0.618807
1  0.002739  0.375628
2  0.739331  0.728760>

In [122]:

### Howmany distinct artists are there in the dataset
Dataset.artist
artists = Dataset['artist']
pd.unique(artists)
len(pd.unique(artists))

3

In [132]:
###  How many artworks by Francis Bacon are there
## Convert to series and then boolean which contains ID and artist value true/false

# S = Dataset['artist'] == 'Blake, William'
# S.value_counts()
# type(S)


### Other way's

artist_count = Dataset['artist'].value_counts()
artist_count['Blake, William']
type(artist_count)


pandas.core.series.Series

In [0]:
### Indexing done the right way
"""
     loc and Iloc to bringall
     LOc by label
     iloc by position
     
    Row labels as Index we set as
    Columnlabel is   0    1
                     1
   selecting by label  df.loc[ "Row Indexer" , "Column Indexer"]  
    Example : df.loc[1035,'artist']
     Also boolean expression df.loc[df['artist']==''Blake, William', :( all columns)]
     
 Selecting by iloc by position
 Example: df.iloc[ row indexer , column indexer
                    100:100     ,  [0,1,4]]
            df.iloc[ row indexer , column indexer
                    :(all rows)     ,  [0,1,4]]        
 
 
"""

In [136]:
####what is the artwork with the biggest dimension

Dataset.loc[1035,'artist']
Dataset.iloc[0,0]
Dataset.iloc[0,:]
Dataset.iloc[0:2,0:2]

'Blake, Robert'

In [0]:
## Try Multiplication
# Dataset['height'] * Dataset['width']
#  Dataset['width'].sort_values().head()
#  Dataset['width'].sort_values().tail()

In [0]:
## Try to convert data object to numeric data types

pd.to_numeric(Dataset['width'])

### Force NAN's
pd.to_numeric(Dataset['width'], errors = 'coerce')
Dataset.loc[:,'width'] = pd.to_numeric(Dataset['width'], errors = 'coerce')



In [146]:
Dataset['units'].value_counts()

mm    50
Name: units, dtype: int64

In [150]:
### create new column with Area

area = Dataset['height'] * Dataset['width']
Dataset = Dataset.assign(area = area)

Dataset.sample(2)

,artist,title,medium,year,acquisitionYear,width,height,units,area
id,,,,,,,,,
1064,"Blake, William",Every Man also Gave him a Piece of Money,Line engraving on paper,1825.0,1919,197,150,mm,29550
1059,"Blake, William",When the Morning Stars Sang Together,Line engraving on paper,1825.0,1919,191,150,mm,28650


In [155]:
Dataset['area'].max()
Dataset['area'].idxmax()  # Index of Maximum row which will display
Dataset.loc[Dataset['area'].idxmax(),:]

artist                                                Blake, William
title              Preliminary Sketch for ‘Christ Girding Himself...
medium                   Graphite on paper. Verso: graphite on paper
year                                                            1805
acquisitionYear                                                 1922
width                                                            505
height                                                           425
units                                                             mm
area                                                          214625
Name: 1077, dtype: object

# Operations on Grouping:

In [0]:
### For missing values with nan
### and filling with most frequesnt data with NAN
### (Transfromation)
## DF.groupby('column') subbset for data frame to small dataframe we can iterate
## and group and further filter data


"""
Iterate over artist Groups
find the date of the first acuistion for each artist
fill some missing values

"""



In [172]:
## Picking small subsetof data
Small_Data = Dataset.iloc[49980:50019,:].copy()
grouped = Small_Data.groupby('artist')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [165]:
for name,group_df in grouped:
    print(name)
    print(group_df)
    break

Frost, Sir Terry
                artist            title               medium  ... width  height units
id                                                            ...                    
4704  Frost, Sir Terry        Blue Moon  Lithograph on paper  ...   355     273    mm
4705  Frost, Sir Terry      Boat Shapes     Linocut on paper  ...   132     143    mm
4706  Frost, Sir Terry      Boat Shapes     Linocut on paper  ...   131     155    mm
4707  Frost, Sir Terry      Boat Shapes     Linocut on paper  ...   193     267    mm
4708  Frost, Sir Terry            Leeds    Drypoint on paper  ...   125     167    mm
4709  Frost, Sir Terry  Camping, Anduze     Etching on paper  ...   257     209    mm
4710  Frost, Sir Terry     Umea, Sweden     Etching on paper  ...   NaN     NaN   NaN
4711  Frost, Sir Terry    Self-Portrait     Etching on paper  ...   277     200    mm

[8 rows x 8 columns]


In [166]:
## Aggregate
## Min

for name,group_df in Small_Data.groupby('artist'):
    min_year = group_df['acquisitionYear'].min()
    print("{}:{}".format(name,min_year))

Frost, Sir Terry:1983.0
Phillips, Esq Tom:1983.0
Wols:1983.0


In [0]:
### Transformation
### Equivelent of editing by hand
### make a case when there is no data to infer
## small_df.loc[[11838,16441],'medium'] = np.nan

def fill_values(series):
    values_counted = series.value_counts()
    if values_counted.empty:
        return series
    most_frequent = values_counted.index[0]
    new_medium    = series.fillna(most_frequent)
    return new_medium



In [0]:
def transform_df(source_df):
    group_dfs = []
    for name,group_df in source_df.groupby('artist'):
        filled_df = group_df.copy()
        filled_df.loc[:,'medium'] = fill_values(group_df['medium'])
        group_dfs.append(filled_df)
        
    new_df = pd.concat(group_dfs)
    return new_df
    

In [0]:
# Small_Data

In [0]:
##Now check the result

filled_df = transform_df(Small_Data)


In [184]:
#### Built in transformations datanwithe  same case above.

### perform aggregation, transformation and 
## filtering using built in methods

grouped_mediums = Small_Data.groupby('artist')['medium']
Small_Data.loc[:,'medium'] = grouped_mediums.transform(fill_values)





,artist,title,medium,year,acquisitionYear,width,height,units
id,,,,,,,,
16412,Wols,[no title],Etching on paper,1937,1983.0,130,80,mm
16413,Wols,[no title],Etching and drypoint on paper,1937,1983.0,187,102,mm
16414,Wols,[no title],Etching and drypoint on paper,1937,1983.0,248,168,mm
16415,Wols,[no title],Etching and drypoint on paper,1937,1983.0,149,102,mm
16416,Wols,[no title],Etching and drypoint on paper,1937,1983.0,203,120,mm
16417,Wols,[no title],Etching on paper,1937,1983.0,130,79,mm
16418,Wols,[no title],Etching and drypoint on paper,1937,1983.0,124,98,mm
16419,Wols,[no title],Etching and drypoint on paper,1937,1983.0,140,89,mm
16420,Wols,[no title],Etching and drypoint on paper,1937,1983.0,140,86,mm


In [188]:
## min
Small_Data.groupby('artist').agg(np.min)
Small_Data.groupby('artist').min()

,title,medium,year,acquisitionYear
artist,,,,
"Frost, Sir Terry",Blue Moon,Drypoint on paper,1952,1983.0
"Phillips, Esq Tom",[colophon],Etching on paper,1979,1983.0
Wols,[no title],Etching and drypoint on paper,1937,1983.0


In [194]:
##filter
grouped_titles = Small_Data.groupby('title')
title_counts = grouped_titles.size().sort_values(ascending=False)
title_counts
## condition

condition = lambda X: len(X.index) > 1
dup_titles_df = grouped_titles.filter(condition)
dup_titles_df.sort_values('title',inplace= True)

dup_titles_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,artist,title,medium,year,acquisitionYear,width,height,units
id,,,,,,,,
4707,"Frost, Sir Terry",Boat Shapes,Linocut on paper,1954,1983.0,193,267,mm
4705,"Frost, Sir Terry",Boat Shapes,Linocut on paper,1952,1983.0,132,143,mm
4706,"Frost, Sir Terry",Boat Shapes,Linocut on paper,1954,1983.0,131,155,mm
16441,Wols,[no title],Etching and drypoint on paper,1937,1983.0,324,248,mm
16440,Wols,[no title],Etching and drypoint on paper,1937,1983.0,105,70,mm
16439,Wols,[no title],Etching and drypoint on paper,1937,1983.0,102,70,mm
16438,Wols,[no title],Etching and drypoint on paper,1937,1983.0,124,98,mm
16437,Wols,[no title],Etching and drypoint on paper,1937,1983.0,124,98,mm
16436,Wols,[no title],Etching and drypoint on paper,1937,1983.0,124,98,mm
